In [26]:
from google.colab import files
import zipfile
import os

# Upload ZIP file
uploaded = files.upload()  # Browse and select the ZIP file

# Extract the uploaded zip file (assuming it's named 'animal_data.zip')
with zipfile.ZipFile("animal_data.zip", 'r') as zip_ref:
    zip_ref.extractall("animal_data_file")  # Extracts to a folder in current directory

print("Extraction complete!")


Saving animal_data.zip to animal_data (2).zip
Extraction complete!


In [67]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt
import os


**1. Problem Statement**

In [ ]:
# To detect animal name via image

**2. Data preprocessing (No augmentation)**

In [68]:
image_size = (224, 224)
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    "animal_data_file/animal_data",
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training",
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    "animal_data_file/animal_data",
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation",
    shuffle=False
)

num_classes = train_generator.num_classes

Found 1561 images belonging to 15 classes.
Found 383 images belonging to 15 classes.


**3. Hypermodel using MobileNet**

In [69]:
def build_cnn_model(hp):
    base_model = ResNet50(include_top=False, weights='imagenet', input_shape=image_size + (3,))
    base_model.trainable = False  # Freeze base

    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())

    units = hp.Int("units", min_value=64, max_value=256, step=32)
    model.add(Dense(
        units=units,
        activation=hp.Choice("activation", ['relu', 'tanh']),
        kernel_initializer=hp.Choice("initializer", ['glorot_uniform', 'he_normal']),
        kernel_regularizer=regularizers.l2(
            hp.Float("lambda", 1e-4, 1e-2, sampling='log'))
    ))

    model.add(Dropout(hp.Float("dropout", 0.2, 0.5, step=0.1)))
    model.add(Dense(num_classes, activation='softmax'))

    learning_rate = hp.Float("learning_rate", 1e-4, 1e-2, sampling='log')
    optimizer_choice = hp.Choice("optimizer", ['adam', 'sgd', 'rmsprop'])

    if optimizer_choice == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate)
    elif optimizer_choice == 'sgd':
        optimizer = tf.keras.optimizers.SGD(learning_rate)
    else:
        optimizer = tf.keras.optimizers.RMSprop(learning_rate)

    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


**4. Keras Tuner Setup**

In [70]:
tuner = kt.RandomSearch(
    build_cnn_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    directory='cnn_tuning',
    project_name='ResNet50_model'
)

early_stop = EarlyStopping(monitor='val_loss', patience=5)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


**5. Start tuning**

In [71]:
tuner.search(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[early_stop],
    verbose=2
)

Trial 10 Complete [00h 01m 30s]
val_loss: 2.5464134216308594

Best val_loss So Far: 2.5464134216308594
Total elapsed time: 00h 15m 48s


**6. Save best model**

In [72]:
best_model = tuner.get_best_models(1)[0]
best_hyperparams = tuner.get_best_hyperparameters(1)[0]

print("\nBest Hyperparameters:")
print(best_hyperparams.values)

if not os.path.exists("model"):
    os.mkdir("model")

best_model.save("model/ResNet50_best_tuned_model.h5")

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))



Best Hyperparameters:
{'units': 128, 'activation': 'tanh', 'initializer': 'glorot_uniform', 'lambda': 0.00011353194581530606, 'dropout': 0.4, 'learning_rate': 0.0003356286726472511, 'optimizer': 'adam'}


**Evaluation**

In [73]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import numpy as np

# Predict the class probabilities
y_pred_train_prob = best_model.predict(train_generator)

# Get the predicted class index (highest probability)
y_pred_train = np.argmax(y_pred_train_prob, axis=1)

# Get the true class labels
y_train = train_generator.classes

# Print evaluation metrics
print("Confusion Matrix:")
print(confusion_matrix(y_train, y_pred_train))

print("\nClassification Report:")
class_labels = list(train_generator.class_indices.keys())  # Get class names
print(classification_report(y_train, y_pred_train, target_names=class_labels))


49/49 ━━━━━━━━━━━━━━━━━━━━ 20s 317ms/step
Confusion Matrix:
[[ 9  0  4 12  0  0 38  0  0  0  0  0  5 31  1]
 [11  0  5  9  0  0 44  0  0  0  0  0  7 30  4]
 [ 9  0  1 14  0  0 34  0  0  0  0  0  3 38  0]
 [ 7  0  1 19  0  0 38  0  0  0  0  0  8 32  0]
 [ 7  1  1 18  0  0 37  0  0  0  0  0  9 28  1]
 [12  0  0 10  0  0 40  0  0  0  0  0  7 28  1]
 [15  1  2 13  0  0 36  0  0  0  0  0  8 28  1]
 [ 9  0  4 15  0  0 44  0  0  0  0  0  4 30  1]
 [ 7  0  2 18  0  0 39  0  0  0  0  0  5 31  2]
 [ 9  0  3 16  0  0 36  0  0  0  0  0 10 30  0]
 [10  0  2 12  0  0 34  0  0  0  0  0  3 38  2]
 [ 6  1  2 18  0  0 47  0  0  0  0  0  6 25  0]
 [16  0  3 11  0  0 34  0  0  0  0  0  5 36  3]
 [ 5  0  0 17  0  0 43  0  0  0  0  0  5 34  0]
 [13  0  2 13  0  0 34  0  0  0  0  0  6 42  0]]

Classification Report:
              precision    recall  f1-score   support

        Bear       0.06      0.09      0.07       100
        Bird       0.00      0.00      0.00       110
         Cat       0.03      0.0

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [74]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    'animal_data_file/animal_data',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


Found 1944 images belonging to 15 classes.


In [75]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import numpy as np

# Step 1: Predict on test data
y_pred_test_prob = best_model.predict(test_generator)

# Step 2: Get the predicted class index (highest probability)
y_pred_test = np.argmax(y_pred_test_prob, axis=1)

# Step 3: Get the true class labels from the test generator
y_test = test_generator.classes

# Step 4: Class labels (same as training)
class_labels = list(test_generator.class_indices.keys())

# Step 5: Print metrics
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_test))

print("\nClassification Report:")
print(classification_report(y_test, y_pred_test, target_names=class_labels))


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


61/61 ━━━━━━━━━━━━━━━━━━━━ 9s 155ms/step
Confusion Matrix:
[[25  0  2 26  0  0 51  0  0  0  0  0  4 17  0]
 [10  2  2 14  0  0 74  0  0  0  0  0  0 34  1]
 [15  0  4 10  0  0 46  0  0  0  0  0  8 40  0]
 [ 6  0  0 56  0  0 48  0  0  0  0  0  5 16  0]
 [14  0  4  8  0  0 49  0  0  0  0  0  0 52  0]
 [18  0  7 28  0  0 43  0  0  0  0  0 16 10  0]
 [ 6  0  4  5  0  0 90  0  0  0  0  0  1 21  2]
 [28  0  1 19  0  0 48  0  0  0  0  0  5 32  0]
 [ 7  0  2  4  0  0 50  0  0  0  0  0  0 66  0]
 [19  0  1 26  0  0 45  0  0  0  0  0  0 37  2]
 [21  1  4 26  0  0 43  0  0  0  0  0  3 27  1]
 [17  0  3 11  0  0 49  0  0  0  0  0 11 40  0]
 [ 0  0  3 12  0  0 31  0  0  0  0  0 74 15  0]
 [ 0  0  0  3  0  0 30  0  0  0  0  0  0 96  0]
 [ 2  0  0 11  0  0 31  0  0  0  0  0  0 76 17]]

Classification Report:
              precision    recall  f1-score   support

        Bear       0.13      0.20      0.16       125
        Bird       0.67      0.01      0.03       137
         Cat       0.11      0.03

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [76]:
print(train_generator.class_indices)


{'Bear': 0, 'Bird': 1, 'Cat': 2, 'Cow': 3, 'Deer': 4, 'Dog': 5, 'Dolphin': 6, 'Elephant': 7, 'Giraffe': 8, 'Horse': 9, 'Kangaroo': 10, 'Lion': 11, 'Panda': 12, 'Tiger': 13, 'Zebra': 14}


In [77]:
train_generator = datagen.flow_from_directory(
    "animal_data_file/animal_data",          # NOT the folder with all images directly inside
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)



Found 1561 images belonging to 15 classes.


In [78]:
val_prob = best_model.predict(val_generator)
val_pred = np.argmax(val_prob, axis=1)
val_true = val_generator.classes

from sklearn.metrics import classification_report
print(classification_report(val_true, val_pred, target_names=class_labels))


12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 438ms/step
              precision    recall  f1-score   support

        Bear       0.19      0.32      0.24        25
        Bird       0.00      0.00      0.00        27
         Cat       0.20      0.04      0.07        24
         Cow       0.16      0.27      0.20        26
        Deer       0.00      0.00      0.00        25
         Dog       0.00      0.00      0.00        24
     Dolphin       0.13      0.80      0.23        25
    Elephant       0.00      0.00      0.00        26
     Giraffe       0.00      0.00      0.00        25
       Horse       0.00      0.00      0.00        26
    Kangaroo       0.00      0.00      0.00        25
        Lion       0.00      0.00      0.00        26
       Panda       0.50      0.67      0.57        27
       Tiger       0.18      0.72      0.29        25
       Zebra       0.86      0.22      0.35        27

    accuracy                           0.20       383
   macro avg       0.15      0.20      

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Evaluate Accuracy on Validation/Test Data

In [79]:
val_loss, val_accuracy = best_model.evaluate(val_generator, verbose=1)

print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
print(f"Validation Loss: {val_loss:.4f}")


12/12 ━━━━━━━━━━━━━━━━━━━━ 10s 322ms/step - accuracy: 0.1649 - loss: 2.5505
Validation Accuracy: 20.37%
Validation Loss: 2.5464
